# Agent with Persistent Memory

This notebook demonstrates how to implement persistent memory in Strands Agents. While the SlidingWindowConversationManager provides conversation context within a single session, persistent memory allows agents to maintain state across multiple sessions.

We'll implement two approaches to persistent memory:
1. File-based persistence
2. Integration with OpenMemory MCP (if available)

## 1. Set up Agent Primitives

In [ ]:
## 1. Model
from strands.models import BedrockModel
bedrock_model = BedrockModel(model_id="us.amazon.nova-pro-v1:0")

## 2. System Prompt / Instructions
system_prompt = f"You are a helpful assistant that provides concise answers. You have access to memory of past interactions. When appropriate, refer to information from previous sessions."

## 3. Standard Conversation Manager (for comparison)
from strands.agent.conversation_manager import SlidingWindowConversationManager
standard_conv_manager = SlidingWindowConversationManager(window_size=4)

## 2. Implement File-Based Persistent Conversation Manager

In [ ]:
import json
import os
from typing import Dict, List, Any, Optional
from strands.agent.conversation_manager import ConversationManager
from strands.types.agent import Message

class FilePersistentConversationManager(ConversationManager):
    """A conversation manager that persists the conversation history to a file."""
    
    def __init__(self, file_path: str, max_messages: int = 100):
        """
        Initialize the persistent conversation manager.
        
        Args:
            file_path: Path to the file where conversation will be stored
            max_messages: Maximum number of messages to keep in memory
        """
        self.file_path = file_path
        self.max_messages = max_messages
        self._messages: List[Message] = []
        
        # Load existing messages if file exists
        self._load_messages()
    
    def _load_messages(self) -> None:
        """Load messages from the persistence file if it exists."""
        if os.path.exists(self.file_path):
            try:
                with open(self.file_path, 'r') as f:
                    data = json.load(f)
                    self._messages = data.get('messages', [])
                    print(f"Loaded {len(self._messages)} messages from {self.file_path}")
            except Exception as e:
                print(f"Error loading messages from {self.file_path}: {e}")
                self._messages = []
    
    def _save_messages(self) -> None:
        """Save messages to the persistence file."""
        try:
            os.makedirs(os.path.dirname(self.file_path), exist_ok=True)
            with open(self.file_path, 'w') as f:
                json.dump({'messages': self._messages}, f, indent=2)
                print(f"Saved {len(self._messages)} messages to {self.file_path}")
        except Exception as e:
            print(f"Error saving messages to {self.file_path}: {e}")
    
    def add_message(self, message: Message) -> None:
        """Add a message to the conversation history and persist it."""
        self._messages.append(message)
        
        # Keep only the last max_messages
        if len(self._messages) > self.max_messages:
            self._messages = self._messages[-self.max_messages:]
        
        # Save to file
        self._save_messages()
    
    def get_messages(self) -> List[Message]:
        """Get all messages in the conversation history."""
        return self._messages
    
    def clear(self) -> None:
        """Clear the conversation history and the persistence file."""
        self._messages = []
        if os.path.exists(self.file_path):
            os.remove(self.file_path)
            print(f"Cleared conversation history and removed {self.file_path}")

# Create a file-based persistent conversation manager
memory_dir = os.path.join(os.getcwd(), "agent_memory")
os.makedirs(memory_dir, exist_ok=True)
file_path = os.path.join(memory_dir, "persistent_memory.json")
persistent_conv_manager = FilePersistentConversationManager(file_path)

## 3. Create Agents with Different Memory Types

In [ ]:
from strands import Agent
from strands_tools import calculator, current_time

# List of tools that our agent will have access to
tool_list = [calculator, current_time]

# Create an agent with standard (non-persistent) memory
standard_agent = Agent(
    model=bedrock_model,
    tools=tool_list,
    system_prompt=system_prompt,
    conversation_manager=standard_conv_manager
)

# Create an agent with persistent memory
persistent_agent = Agent(
    model=bedrock_model,
    tools=tool_list,
    system_prompt=system_prompt,
    conversation_manager=persistent_conv_manager
)

## 4. Test Persistent Memory

Let's use our persistent agent and see how it maintains state across interactions.

In [ ]:
# First interaction
response = persistent_agent("My name is Alice and I like machine learning and quantum computing")
print(f"Agent response: {response.message}")

In [ ]:
# Second interaction
response = persistent_agent("What's the current time?")
print(f"Agent response: {response.message}")

In [ ]:
# Let's check our conversation history
print("Conversation history:")
for idx, msg in enumerate(persistent_agent.messages):
    role = "User" if msg["role"] == "user" else "Assistant"
    print(f"{idx+1}. {role}: {msg['content']}")

## 5. Simulate a New Session

Now let's simulate starting a new session by creating a new agent instance with the same memory file. This demonstrates how the agent can access memory across different sessions.

In [ ]:
# Create a new agent with the same persistent memory file
new_session_agent = Agent(
    model=bedrock_model,
    tools=tool_list,
    system_prompt=system_prompt,
    conversation_manager=FilePersistentConversationManager(file_path)
)

# Check if the new agent has the memory from previous session
print("New session agent memory:")
for idx, msg in enumerate(new_session_agent.messages):
    role = "User" if msg["role"] == "user" else "Assistant"
    print(f"{idx+1}. {role}: {msg['content']}")

In [ ]:
# Test if the agent remembers information from the previous session
response = new_session_agent("What's my name and what are my interests?")
print(f"Agent response: {response.message}")

## 6. Enhanced Memory Management with Tagging and Recall

Let's extend our memory system to include tagging for better organization and retrieval.

In [ ]:
import re
from datetime import datetime

class EnhancedMemoryManager(FilePersistentConversationManager):
    """Enhanced memory manager with tagging and recall capabilities."""
    
    def __init__(self, file_path: str, max_messages: int = 100):
        super().__init__(file_path, max_messages)
        self.memory_tags = {}
        self._load_tags()
    
    def _load_tags(self) -> None:
        """Load memory tags from the persistence file."""
        if os.path.exists(self.file_path):
            try:
                with open(self.file_path, 'r') as f:
                    data = json.load(f)
                    self.memory_tags = data.get('tags', {})
            except Exception as e:
                print(f"Error loading tags from {self.file_path}: {e}")
                self.memory_tags = {}
    
    def _save_messages(self) -> None:
        """Save messages and tags to the persistence file."""
        try:
            os.makedirs(os.path.dirname(self.file_path), exist_ok=True)
            with open(self.file_path, 'w') as f:
                json.dump({
                    'messages': self._messages,
                    'tags': self.memory_tags
                }, f, indent=2)
        except Exception as e:
            print(f"Error saving to {self.file_path}: {e}")
    
    def add_memory_tag(self, tag: str, message_idx: int) -> None:
        """Add a tag to a specific message in memory."""
        if tag not in self.memory_tags:
            self.memory_tags[tag] = []
            
        if message_idx not in self.memory_tags[tag]:
            self.memory_tags[tag].append(message_idx)
            self._save_messages()
    
    def auto_tag(self, message: Message) -> None:
        """Automatically tag a message based on its content."""
        content = message.get('content', '')
        message_idx = len(self._messages) - 1
        
        # Example auto-tagging rules
        if re.search(r'\bname\b', content, re.IGNORECASE):
            self.add_memory_tag('personal_info', message_idx)
            
        if re.search(r'\blike\b|\binterest\b|\bhobby\b', content, re.IGNORECASE):
            self.add_memory_tag('preferences', message_idx)
            
        # Add timestamp tag
        timestamp = datetime.now().strftime('%Y-%m-%d')
        self.add_memory_tag(f'date_{timestamp}', message_idx)
    
    def add_message(self, message: Message) -> None:
        """Add a message to memory with auto-tagging."""
        super().add_message(message)
        if message.get('role') == 'user':
            self.auto_tag(message)
    
    def recall_by_tag(self, tag: str) -> List[Message]:
        """Retrieve messages by tag."""
        if tag not in self.memory_tags:
            return []
            
        return [self._messages[idx] for idx in self.memory_tags[tag] 
                if 0 <= idx < len(self._messages)]

# Create an agent with enhanced memory
enhanced_memory_path = os.path.join(memory_dir, "enhanced_memory.json")
enhanced_memory_manager = EnhancedMemoryManager(enhanced_memory_path)

enhanced_agent = Agent(
    model=bedrock_model,
    tools=tool_list,
    system_prompt=system_prompt + " You can recall information by topics like personal_info or preferences.",
    conversation_manager=enhanced_memory_manager
)

In [ ]:
# Test the enhanced memory agent
response = enhanced_agent("My name is Bob and I enjoy hiking, photography, and reading science fiction books.")
print(f"Agent response: {response.message}")

In [ ]:
# Add a tool for memory recall
from strands.tools import tool

@tool
def recall_memory(tag: str) -> str:
    """Recall information from memory based on a tag."""
    messages = enhanced_memory_manager.recall_by_tag(tag)
    if not messages:
        return f"No memories found with tag '{tag}'"
        
    results = []
    for msg in messages:
        role = "User" if msg["role"] == "user" else "Assistant"
        results.append(f"{role}: {msg['content']}")
        
    return "\n".join(results)

# Update the enhanced agent with the recall tool
enhanced_agent.tools = tool_list + [recall_memory]

# Test recall
response = enhanced_agent("What do you know about my interests?")
print(f"Agent response: {response.message}")

## 7. Integration with OpenMemory MCP (Optional)

If you're running this with OpenMemory MCP server, uncomment the following code to integrate with it.

In [ ]:
# Uncomment if using OpenMemory MCP

# try:
#     from mem0.client import MemoryClient
#     from strands.agent.conversation_manager import ConversationManager
# 
#     class OpenMemoryManager(ConversationManager):
#         """Conversation manager that uses OpenMemory for persistence."""
#         
#         def __init__(self, memory_id: str = "strands-agent-memory"):
#             """
#             Initialize the OpenMemory conversation manager.
#             
#             Args:
#                 memory_id: Identifier for this memory instance
#             """
#             self.memory_id = memory_id
#             self.client = MemoryClient()
#             self._messages = []
#             self._load_messages()
#         
#         def _load_messages(self) -> None:
#             """Load messages from OpenMemory."""
#             try:
#                 memory_data = self.client.get_memory(self.memory_id)
#                 if memory_data and 'messages' in memory_data:
#                     self._messages = memory_data['messages']
#             except Exception as e:
#                 print(f"Error loading from OpenMemory: {e}")
#                 self._messages = []
#         
#         def _save_messages(self) -> None:
#             """Save messages to OpenMemory."""
#             try:
#                 self.client.store_memory(self.memory_id, {'messages': self._messages})
#             except Exception as e:
#                 print(f"Error saving to OpenMemory: {e}")
#         
#         def add_message(self, message: Message) -> None:
#             """Add a message to the conversation history and persist it."""
#             self._messages.append(message)
#             self._save_messages()
#         
#         def get_messages(self) -> List[Message]:
#             """Get all messages in the conversation history."""
#             return self._messages
#         
#         def clear(self) -> None:
#             """Clear the conversation history."""
#             self._messages = []
#             self.client.delete_memory(self.memory_id)
# 
#     # Create an agent with OpenMemory integration
#     openmemory_manager = OpenMemoryManager()
#     openmemory_agent = Agent(
#         model=bedrock_model,
#         tools=tool_list,
#         system_prompt=system_prompt,
#         conversation_manager=openmemory_manager
#     )
#     
#     print("OpenMemory integration available")
# except ImportError:
#     print("OpenMemory integration not available. Install mem0-client to use this feature.")

## 8. Memory Analysis and Management

Here's a helper function to analyze memory usage and content.

In [ ]:
def analyze_memory(conversation_manager):
    """Analyze memory content and usage."""
    messages = conversation_manager.get_messages()
    
    # Basic stats
    total_messages = len(messages)
    user_messages = sum(1 for msg in messages if msg.get('role') == 'user')
    assistant_messages = sum(1 for msg in messages if msg.get('role') == 'assistant')
    
    print(f"Memory Analysis:")
    print(f"- Total messages: {total_messages}")
    print(f"- User messages: {user_messages}")
    print(f"- Assistant messages: {assistant_messages}")
    
    # Tag analysis for enhanced memory
    if hasattr(conversation_manager, 'memory_tags'):
        print("\nMemory Tags:")
        for tag, indices in conversation_manager.memory_tags.items():
            print(f"- {tag}: {len(indices)} messages")
    
    # Memory file size
    if hasattr(conversation_manager, 'file_path') and os.path.exists(conversation_manager.file_path):
        file_size = os.path.getsize(conversation_manager.file_path) / 1024  # KB
        print(f"\nMemory file size: {file_size:.2f} KB")

# Analyze the different memory implementations
print("Standard Memory (SlidingWindowConversationManager):")
analyze_memory(standard_conv_manager)

print("\nPersistent Memory (FilePersistentConversationManager):")
analyze_memory(persistent_conv_manager)

print("\nEnhanced Memory (EnhancedMemoryManager):")
analyze_memory(enhanced_memory_manager)

## 9. Memory Cleanup

If you want to reset the memory, you can use the clear method. Uncomment and run when needed.

In [ ]:
# Uncomment to clear memory
# persistent_conv_manager.clear()
# enhanced_memory_manager.clear()
# if 'openmemory_manager' in locals():
#     openmemory_manager.clear()

## 10. Conclusion

This notebook demonstrated several approaches to implementing persistent memory in Strands Agents:

1. **File-based Persistence**: Simple storage of conversation history in a JSON file
2. **Enhanced Memory with Tagging**: Adding semantic organization through auto-tagging
3. **Memory Recall Tools**: Providing agents with tools to access specific memories
4. **OpenMemory Integration**: (Optional) Using the OpenMemory MCP for more advanced memory capabilities

Persistent memory enables agents to maintain context across multiple sessions, creating more personalized and context-aware interactions. This is particularly valuable for long-running agent applications and user-specific customization.